<a href="https://colab.research.google.com/github/isabelklint/scrapers/blob/main/MEREJA_WEB_SCRAPER_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://github.com/tilburgsciencehub/data-ranking-the-brands/blob/main/webscraping_ranking_the_brands.ipynb
# !pip install BeautifulSoup4
# !pip install pandas
# !pip install requests

from bs4 import BeautifulSoup
import requests
import pandas as pd


In [ ]:
# chose a value for the while loop, here it is 2 so you can see how it works
i = 0
soups = []
while i < 2: # 50 is up to september 2022
  page_link = 'https://mereja.com/amharic/v2/page/' + str(i)
  request_object = requests.get(page_link)
  source_code = request_object.text
  soup = BeautifulSoup(source_code, "html.parser")
  soups.append(soup)
  i+=1


In [ ]:
# make some lists
list_of_URLS = []
scrapedURLS = []

In [ ]:
# get all the links
for soup in soups:
  for span in soup.find_all('a'):
    list_of_URLS.append(span.get('href'))
    

In [ ]:
# you can copy and paste the soups into vs code 
# or similar and look at the structure of the html
# so you know what tags etc. to target

#print(soups)

In [ ]:
# get a list of urls that make sense for news items in amaric
# these won't include videos tho
# video url: https://mereja.com/video2/watch.php?vid=f94162ab5
for url in list_of_URLS:
  if url.startswith('https://mereja.com/amharic/v2/8') == True:
    scrapedURLS.append(url)
  

In [ ]:
# how many urls?
len(scrapedURLS)

168

In [ ]:
# get rid of duplicates
unique_list = list(set(scrapedURLS))
print(len(unique_list))

84


In [ ]:
# set name again
scrapedURLS = unique_list

In [ ]:
# an example url
print(scrapedURLS[10])

https://mereja.com/amharic/v2/812791


In [ ]:
# adapted from https://blog.devgenius.io/use-selenium-webdriver-in-google-colab-d5f2dba1d9f5
#First, we install cloudscraper and googletrans modules that we are going to use for scraping and translating the content.

!pip install cloudscraper
!pip install google_trans_new


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 KB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 KB 6.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#With cloudscraper we scrape the URL and with Beutiful Soup we parse the URLs.
#We put everything together in order based on importance.
import cloudscraper
from bs4 import BeautifulSoup


In [ ]:
# here is the specific function to the webpage, 
# this one is specific to mereja's tags
data = pd.DataFrame(columns = ['Date', 'Title', 'URL'])

def scrape_this_url(url):
    scraper = cloudscraper.create_scraper() 
    headers = {'user-agent': 'Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Build/MMB29P) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Mobile Safari/537.36 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)'}
    
    try:
        r = scraper.get(url, headers=headers)
        soup = BeautifulSoup(r.text, 'html.parser')
        title = soup.find('title').text
        span = soup.find('span', {'class': 'timestamp updated'}).text
        # create list of readmore links (is there more than one per page?)
        # linked_tag = soup.find('div', {'class': 'readmore'})
        # linked = linked_tag.find('a').get('href') if linked_tag and linked_tag.find('a') else ""
        linked_tags = soup.find_all('div', {'class': 'readmore'})
        linked = []
        for tag in linked_tags:
            link = tag.find('a').get('href') if tag.find('a') else ""
            linked.append(link)

        # Create a dictionary with the data to add
        data_dict = {'Date': span, 'Title': title, 'URL': url, 'Linked_URL': linked}

        # return the dictionary
        return data_dict

    except Exception as e:
        print(e)
        return {}


In [ ]:
# i do not recommend this but the output was too cluttered
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")


In [ ]:
# loop through the list of URLs and append the data to the DataFrame
for url in scrapedURLS:
    data_dict = scrape_this_url(url)
    if data_dict:
        data = data.append(data_dict, ignore_index=True)


In [ ]:
# take a look at output
data.head()


,Date,Title,URL,Linked_URL
0,"March 31, 2023",ሩሲያ ምግብ ለሰሜን ኮሪያ ሰጥታ በምላሹ ጦር መሳሪያ ልትቀበል መሆኑን አ...,https://mereja.com/amharic/v2/811704,https://www.bbc.com/amharic/articles/crgjed2vy...
1,"April 3, 2023",የሽግግር ሂደት በትግራይ፤ ተስፋና ስጋቶቹ | Mereja.com - Ethi...,https://mereja.com/amharic/v2/812588,https://www.dw.com/am/%E1%8B%A8%E1%88%BD%E1%8C...
2,"March 31, 2023",“ኡጋንዳውያኑ ወደ ኢትዮጵያ የገቡት የምጽዓትን ቀን ሽሽት አይደለም” የኛ...,https://mereja.com/amharic/v2/811842,https://www.bbc.com/amharic/articles/cq5ew7xnp...
3,"March 31, 2023",የአላማጣ ከተማ ወቅታዊ ሁኔታ እና የነዋሪዎች ቅሬታ | Mereja.com ...,https://mereja.com/amharic/v2/811719,https://www.bbc.com/amharic/articles/ckkqqexrz...
4,"March 31, 2023",ሩሲያ ምግብ ለሰሜን ኮሪያ ሰጥታ በምላሹ ጦር መሳሪያ ልትቀበል መሆኑን አ...,https://mereja.com/amharic/v2/811703,https://www.bbc.com/amharic/articles/crgjed2vy...


In [ ]:
# get some stats on output
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110 entries, 0 to 109
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Date        110 non-null    object
 1   Title       110 non-null    object
 2   URL         110 non-null    object
 3   Linked_URL  110 non-null    object
dtypes: object(4)
memory usage: 3.6+ KB


In [30]:
# get some stats on output
data.describe()

,Date,Title,URL,Linked_URL
count,80,80,80,80
unique,6,80,80,31
top,"April 2, 2023",በሸገር ከተማ ቤቶች መፍረስ | Mereja.com - Ethiopian Amh...,https://mereja.com/amharic/v2/812321,[]
freq,20,1,1,50


In [ ]:
# # filter the DataFrame by URLs containing "video2"
# filtered_df = data[data['URL'].str.contains('video2')]

# # print the filtered DataFrame
# print(filtered_df)